## Speed dating data-set
#### Joris Rombouts - November 2017

### 1.3 Feature Engineering

### <font color="green">imports, preparation and configuration</font>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import tree, preprocessing, metrics
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("speed_dating_assignment.csv")
df.head()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


In [3]:
#df['age_o'] = np.nan
df_o = df.copy()
df_o = df_o.filter(items=['iid', 'age', 'race', 'from', 'goal'])
df_o.rename(columns={'iid': 'pid', 'age': 'age_o', 'race': 'race_partner', 'from': 'from_partner', 'goal':'goal_partner'}, inplace = True)
df_o = df_o.drop_duplicates()
df_new = pd.merge(df, df_o, on=['pid'], how = 'left')
df_new.head()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3,age_o,race_partner,from_partner,goal_partner
0,1,1.0,0,1,1,1,10,7,NaN,4,...,7.0,NaN,NaN,NaN,NaN,NaN,27.0,2.0,Argentina,1.0
1,1,1.0,0,1,1,1,10,7,NaN,3,...,7.0,NaN,NaN,NaN,NaN,NaN,22.0,2.0,England,1.0
2,1,1.0,0,1,1,1,10,7,NaN,10,...,7.0,NaN,NaN,NaN,NaN,NaN,22.0,4.0,Southern California,2.0
3,1,1.0,0,1,1,1,10,7,NaN,5,...,7.0,NaN,NaN,NaN,NaN,NaN,23.0,2.0,"Cincinnati, Ohio",2.0
4,1,1.0,0,1,1,1,10,7,NaN,7,...,7.0,NaN,NaN,NaN,NaN,NaN,24.0,3.0,Miami,1.0


In [4]:
df =df_new.copy()
df.head()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3,age_o,race_partner,from_partner,goal_partner
0,1,1.0,0,1,1,1,10,7,NaN,4,...,7.0,NaN,NaN,NaN,NaN,NaN,27.0,2.0,Argentina,1.0
1,1,1.0,0,1,1,1,10,7,NaN,3,...,7.0,NaN,NaN,NaN,NaN,NaN,22.0,2.0,England,1.0
2,1,1.0,0,1,1,1,10,7,NaN,10,...,7.0,NaN,NaN,NaN,NaN,NaN,22.0,4.0,Southern California,2.0
3,1,1.0,0,1,1,1,10,7,NaN,5,...,7.0,NaN,NaN,NaN,NaN,NaN,23.0,2.0,"Cincinnati, Ohio",2.0
4,1,1.0,0,1,1,1,10,7,NaN,7,...,7.0,NaN,NaN,NaN,NaN,NaN,24.0,3.0,Miami,1.0


In [5]:
#df_new = df[(df['wave'] >= 6) & (df['wave'] <= 9)]
#df_new.shape

In [6]:
#df = pd.concat([df, df_new, df_new]).drop_duplicates(keep=False)
#df.head()

In [7]:
df = df[['attr', 'sinc', 'intel', 'fun', 'amb', 'shar', 'like', 'gender', 'order','goal', 'field_cd', 'date','age', 'go_out', 'imprace', 'imprelig', 'dec', 'age_o', 'from', 'goal', 'race', 'pid', 'iid', 'race_partner', 'from_partner', 'goal_partner']]
df.age.fillna(df.age.median(), inplace = True)
df.head()

,attr,sinc,intel,fun,amb,shar,like,gender,order,goal,...,dec,age_o,from,goal,race,pid,iid,race_partner,from_partner,goal_partner
0,6.0,9.0,7.0,7.0,6.0,5.0,7.0,0,4,2.0,...,1,27.0,Chicago,2.0,4.0,11,1,2.0,Argentina,1.0
1,7.0,8.0,7.0,8.0,5.0,6.0,7.0,0,3,2.0,...,1,22.0,Chicago,2.0,4.0,12,1,2.0,England,1.0
2,5.0,8.0,9.0,8.0,5.0,7.0,7.0,0,10,2.0,...,1,22.0,Chicago,2.0,4.0,13,1,4.0,Southern California,2.0
3,7.0,6.0,8.0,7.0,6.0,8.0,7.0,0,5,2.0,...,1,23.0,Chicago,2.0,4.0,14,1,2.0,"Cincinnati, Ohio",2.0
4,5.0,6.0,7.0,7.0,6.0,6.0,6.0,0,7,2.0,...,1,24.0,Chicago,2.0,4.0,15,1,3.0,Miami,1.0


In [8]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8378 entries, 0 to 8377
Data columns (total 26 columns):
attr            8176 non-null float64
sinc            8101 non-null float64
intel           8082 non-null float64
fun             8028 non-null float64
amb             7666 non-null float64
shar            7311 non-null float64
like            8138 non-null float64
gender          8378 non-null int64
order           8378 non-null int64
goal            8299 non-null float64
field_cd        8296 non-null float64
date            8281 non-null float64
age             8283 non-null float64
go_out          8299 non-null float64
imprace         8299 non-null float64
imprelig        8299 non-null float64
dec             8378 non-null int64
age_o           8274 non-null float64
from            8299 non-null object
goal            8299 non-null float64
race            8315 non-null float64
pid             8368 non-null object
iid             8378 non-null int64
race_partner    8305 non-null

In [9]:
df = df.dropna().reset_index(drop=True)

In [10]:
df['attr_o'] = df['attr'] + df['sinc'] + df['intel'] + df['fun'] + df['shar'] + df['like'] + df['amb']
df = df.drop(['attr', 'sinc', 'intel', 'fun', 'shar', 'like', 'amb'], axis = 1)
df.head()

,gender,order,goal,field_cd,date,age,go_out,imprace,imprelig,dec,age_o,from,goal,race,pid,iid,race_partner,from_partner,goal_partner,attr_o
0,0,4,2.0,1.0,7.0,21.0,1.0,2.0,4.0,1,27.0,Chicago,2.0,4.0,11,1,2.0,Argentina,1.0,47.0
1,0,3,2.0,1.0,7.0,21.0,1.0,2.0,4.0,1,22.0,Chicago,2.0,4.0,12,1,2.0,England,1.0,48.0
2,0,10,2.0,1.0,7.0,21.0,1.0,2.0,4.0,1,22.0,Chicago,2.0,4.0,13,1,4.0,Southern California,2.0,49.0
3,0,5,2.0,1.0,7.0,21.0,1.0,2.0,4.0,1,23.0,Chicago,2.0,4.0,14,1,2.0,"Cincinnati, Ohio",2.0,49.0
4,0,7,2.0,1.0,7.0,21.0,1.0,2.0,4.0,1,24.0,Chicago,2.0,4.0,15,1,3.0,Miami,1.0,43.0


In [11]:
df['age_diff'] = abs(df['age'] - df['age_o'])
df.head()

,gender,order,goal,field_cd,date,age,go_out,imprace,imprelig,dec,...,from,goal,race,pid,iid,race_partner,from_partner,goal_partner,attr_o,age_diff
0,0,4,2.0,1.0,7.0,21.0,1.0,2.0,4.0,1,...,Chicago,2.0,4.0,11,1,2.0,Argentina,1.0,47.0,6.0
1,0,3,2.0,1.0,7.0,21.0,1.0,2.0,4.0,1,...,Chicago,2.0,4.0,12,1,2.0,England,1.0,48.0,1.0
2,0,10,2.0,1.0,7.0,21.0,1.0,2.0,4.0,1,...,Chicago,2.0,4.0,13,1,4.0,Southern California,2.0,49.0,1.0
3,0,5,2.0,1.0,7.0,21.0,1.0,2.0,4.0,1,...,Chicago,2.0,4.0,14,1,2.0,"Cincinnati, Ohio",2.0,49.0,2.0
4,0,7,2.0,1.0,7.0,21.0,1.0,2.0,4.0,1,...,Chicago,2.0,4.0,15,1,3.0,Miami,1.0,43.0,3.0


In [17]:
df['race_diff'] = (df['race'] == df['race_partner'])
df['from_equals'] = (df['from'] == df['from_partner'])

### train models

In [ ]:
male_subjects = df[df.gender == 1]
female_subjects = df[df.gender == 0]
male_subjects.shape, female_subjects.shape

In [ ]:
male_subjects_shuffle = male_subjects.sample(frac=1).reset_index(drop=True) # shuffle rows
female_subjects_shuffle = female_subjects.sample(frac=1).reset_index(drop=True)

In [ ]:
male_x_data = male_subjects_shuffle.drop('dec', axis = 1) # dec is target attribute
female_x_data = female_subjects_shuffle.drop('dec', axis = 1)
male_labels = male_subjects_shuffle['dec']
female_labels = female_subjects_shuffle['dec']

In [ ]:
male_x_train, male_x_test, male_y_train, male_y_test = train_test_split(male_x_data, male_labels, test_size = 0.2)
female_x_train, female_x_test, female_y_train, female_y_test = train_test_split(female_x_data, female_labels, test_size = 0.2)

In [ ]:
#np.all(np.isfinite(df))

In [ ]:
#np.any(np.isnan(df))

In [ ]:
dec_tree_male = tree.DecisionTreeClassifier(criterion = "gini", max_depth = 8, min_samples_split = 50, min_samples_leaf = 12)
dec_tree_female = tree.DecisionTreeClassifier(criterion = "gini", max_depth = 8, min_samples_split = 50, min_samples_leaf = 12)
# build decision tree classifiers from the training sets
dec_tree_male.fit(male_x_train, male_y_train)
dec_tree_female.fit(female_x_train, female_y_train)

In [ ]:
pred_male = dec_tree_male.predict(male_x_test) # predicting 'dec' for data which the model has not seen yet
print("Accuracy score: ", metrics.accuracy_score(male_y_test, pred_male))

In [ ]:
pred_female = dec_tree_female.predict(female_x_test) # predicting 'dec' for data which the model has not seen yet
print("Accuracy score: ", metrics.accuracy_score(female_y_test, pred_female))